In [98]:
import respy as rp
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sqlalchemy as db

from estimagic.optimization.utilities import *

In [99]:
params_corr, options, df = rp.get_example_model("robinson_crusoe_basic")

In [117]:
infos = params_corr.filter(like="sd_", axis=0).index.get_level_values(1).to_list()
choices = list()
for info in infos:
    choices.append(info.split("_")[1])

final = list()
for outer in choices:
    for inner in choices:
    
        if inner == outer:
            final.append('chol_' + outer)
        else:
            final.append('chol_' + outer + "_" + inner)
        
        if inner == outer:
            break

 
rename_dict = dict()
for key_ in params_corr.loc[("shocks_sdcorr", slice(None))].index:
    rename_dict[key_] = final.pop(0)
rename_dict
        

{'sd_fishing': 'chol_fishing',
 'sd_hammock': 'chol_hammock_fishing',
 'corr_hammock_fishing': 'chol_hammock'}

In [118]:

num_dim = params_corr.filter(like="sd_", axis=0).shape[0]

sds, corr = sdcorr_params_to_sds_and_corr(params_corr.loc[("shocks_sdcorr", slice(None)), "value"])
rslt = robust_cholesky(sds_and_corr_to_cov(sds, corr))
params_chol = params_corr.rename(index={"shocks_sdcorr": 'shocks_chol'})
params_chol.loc[("shocks_chol", slice(None)), "value"] = rslt[np.tril_indices(num_dim)] 

params_chol.rename(index=rename_dict, inplace=True)
sds_and_corr_to_cov(sds, corr)

array([[ 1. , -0.2],
       [-0.2,  1. ]])

In [120]:
crit_func = rp.get_crit_func(params_corr, options, df)
np.testing.assert_equal(crit_func(params_corr), crit_func(params_chol))